In [10]:
#Here we import all the necessary dependencies
import numpy as np
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output
from amuse.lab import units

In [18]:
#Here we generate a basic solarsystem, with only the gas giants
from amuse.ext.solarsystem import new_solar_system

def create_system():
    
    system = new_solar_system()
    system = system[system.mass > 10**-5 | units.MSun]
    system.move_to_center()
    return system
    
    
basic_giants_system = create_system()

|S11


In [ ]:
#Here we add the Oort cloud objects, according to a chosen distribution
def random_position(): #There is probably some AMUSE function for this
    
def random_velocity(): #There is probably some AMUSE function for this

def add_oort_objects(system, N_objects):
    
    for i in range(N_objects):
        oort = Particles(1)
        oort.name = "OORT_" + str(i)
        oort.mass = 0.0 | units.MSun
        oort.position =  random_position() | units.AU #Take eccentricity + inclination + semi-major axis according to some dist.
        oort.velocity =  random_velocity() | units.kms

        system.add_particle(oort)
    return system

complete_system = add_oort_objects(basic_giants_system, N_objects = )

In [19]:
#Here we generate a galactic potential 

class MilkyWay_galaxy(object):
    def __init__(self, 
                 Mb=1.40592e10| units.MSun,
                 Md=8.5608e10| units.MSun,
                 Mh=1.07068e11 | units.MSun):
        self.Mb= Mb
        self.Md= Md
        self.Mh= Mh

    def get_potential_at_point(self,eps,x,y,z):
        r=(x**2+y**2+z**2)**0.5
        R= (x**2+y**2)**0.5
        # buldge
        b1= 0.3873 |units.kpc
        pot_bulge= -constants.G*self.Mb/(r**2+b1**2)**0.5 
        # disk
        a2= 5.31 |units.kpc
        b2= 0.25 |units.kpc
        pot_disk = -constants.G*self.Md/(R**2 + (a2+ (z**2+ b2**2)**0.5 )**2 )**0.5
        #halo
        a3= 12.0 |units.kpc
        cut_off=100 |units.kpc
        d1= r/a3
        c=1+ (cut_off/a3)**1.02
        pot_halo= -constants.G*(self.Mh/a3)*d1**1.02/(1+ d1**1.02) \
                  - (constants.G*self.Mh/(1.02*a3))\
                      * (-1.02/c +numpy.log(c) + 1.02/(1+d1**1.02) \
                           - numpy.log(1.0 +d1**1.02) )
        return 2*(pot_bulge+pot_disk+ pot_halo) 
                # multiply by 2 because it is a rigid potential
    
    def get_gravity_at_point(self, eps, x,y,z): 
        r= (x**2+y**2+z**2)**0.5
        R= (x**2+y**2)**0.5
        #bulge
        b1= 0.3873 |units.kpc
        force_bulge= -constants.G*self.Mb/(r**2+b1**2)**1.5 
        #disk
        a2= 5.31 |units.kpc
        b2= 0.25 |units.kpc
        d= a2+ (z**2+ b2**2)**0.5
        force_disk=-constants.G*self.Md/(R**2+ d**2 )**1.5
        #halo
        a3= 12.0 |units.kpc
        d1= r/a3
        force_halo= -constants.G*self.Mh*d1**0.02/(a3**2*(1+d1**1.02))
       
        ax= force_bulge*x + force_disk*x  + force_halo*x/r
        ay= force_bulge*y + force_disk*y  + force_halo*y/r
        az= force_bulge*z + force_disk*d*z/(z**2 + b2**2)**0.5 + force_halo*z/r 

        return ax,ay,az
    
MW_potential = MilkyWay_galaxy()

In [ ]:
#Here we create the conditions for the migration of the planets

def planet_migration():
    
    velocities = 
    return velocities

final_system = complete_system.add(velocities)

In [ ]:
#Here we perform the conversion for the system

final_converter=nbody_system.nbody_to_si(final_system.mass.sum(), #CHECK IF THIS IS CORRECT
                                   final_system.position.length())

In [ ]:
#Here we evolve the system, also plotting the spatial evolution of the objects
from amuse.community.ph4.interface import ph4

def evolver(particle_system, potential, converter, end_time=4*10**3, time_step=0.1):
    
    gravity_code = ph4(converter)
    gravity_code.particles.add_particles(particle_system)
    ch_g2l = gravity_code.particles.new_channel_to(particle_system)
    
    gravity_bridge = 0
    gravity_bridge = bridge.Bridge(use_threading=False)
    gravity_bridge.add_system(gravity_code, (potential,) )
    gravity_bridge.timestep = time_step|units.Myr

    times = numpy.arange(0., end_time, time_step) | units.Myr
    for time in times:
        gravity_bridge.evolve_model(time)
        ch_g2l.copy()
    gravity.stop()
    
    
    
evolved_system = evolver(final_system, MW_potential, final_converter end_time=, time_step=)